In [9]:
import ee
import geemap
import datetime
import numpy as np

In [10]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')

modis_date_start = ee.Date("2001-01-01") # в 2001 начал предоставлять тепловую информацию Terra
landsat_date_start = ee.Date("1986-03-16") # с 1986 начал предоставлять тепловую информацию Landsat-5
landsat_start_year = landsat_date_start.get("year").getInfo()
modis_start_year = modis_date_start.get("year").getInfo()
date_end = datetime.datetime.now()
end_year = date_end.date().year
openwater_month_start = 6 # июнь
openwater_month_end = 11 # ноябрь

baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

# LANDSAT

In [11]:
LandsatLST = geemap.requireJS("../baikal_vkr_init/Landsat_LST.js")

In [12]:
use_ndvi = True

def get_landsat_collection(satellite_name: str):
   coll = LandsatLST.collection(satellite_name, landsat_date_start, date_end, baikal_shape, use_ndvi)#.filter(ee.Filter.lt('CLOUD_COVER', 30))
   return coll.select(['LST'])

In [ ]:
landsats_collection = ee.ImageCollection.merge(get_landsat_collection('L8'),get_landsat_collection('L9'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L7'))
# landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L5'))

In [ ]:
def get_average_lst_landsat(landsat_collection: ee.ImageCollection, start_time, end_time):
   temp = landsat_collection.filterDate(start_time, end_time).mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return result_dict['LST']

In [13]:
lst_landsat = {}

for year in range(landsat_start_year, end_year):
   lst_landsat[year] = {}
   for month in range(openwater_month_start, openwater_month_end):
      start_date = ee.Date(f"{year}-{month}-1")
      end_date = ee.Date(f"{year}-{(month+1)}-1")
      
      result = get_average_lst_landsat(landsats_collection, start_date, end_date)
      if(result is None):
         continue
      lst_landsat[year][month] = result
   
   lst_landsat[year] = np.mean(list(lst_landsat[year].values()))
   if lst_landsat[year] is None:
      del lst_landsat[year]

lst_landsat

{1996: 7.615046916830711,
 1997: 6.4652446881016505,
 1998: 8.29735257295523,
 1999: 7.466276454270664,
 2000: 8.444648433179971,
 2001: 6.8573912127145835,
 2002: 11.29452174088983,
 2003: 8.133507938438473,
 2004: 7.756460249341295,
 2005: 8.815491983317168,
 2006: 6.796633766422254,
 2007: 9.983745663150753,
 2008: 8.444456981800055,
 2009: 7.979558202903978,
 2010: 7.409646087737466,
 2011: 7.802575171575934,
 2012: 8.998809616805136,
 2013: 9.029895691880107,
 2014: 10.268777774431134,
 2015: 10.722652392625395,
 2016: 10.075703142279956,
 2017: 9.112241223415538,
 2018: 9.1208041147658,
 2019: 9.838905106835822,
 2020: 10.8137280883284,
 2021: 8.059809265577483,
 2022: 8.255228706723798,
 2023: 9.533078486705547,
 2024: 9.525957321098034}

In [ ]:
del lst_landsat[1983]
del lst_landsat[1984]
del lst_landsat[1985]

In [ ]:
lst_landsat_s = { 1996: 7.615046916830711,
 1997: 6.4652446881016505,
 1998: 8.29735257295523,
 1999: 7.466276454270664,
 2000: 8.444648433179971,
 2001: 6.8573912127145835,
 2002: 11.29452174088983,
 2003: 8.133507938438473,
 2004: 7.756460249341295,
 2005: 8.815491983317168,
 2006: 6.796633766422254,
 2007: 9.983745663150753,
 2008: 8.444456981800055,
 2009: 7.979558202903978,
 2010: 7.409646087737466,
 2011: 7.802575171575934,
 2012: 8.998809616805136,
 2013: 9.029895691880107,
 2014: 10.268777774431134,
 2015: 10.722652392625395,
 2016: 10.075703142279956,
 2017: 9.112241223415538,
 2018: 9.1208041147658,
 2019: 9.838905106835822,
 2020: 10.8137280883284,
 2021: 8.059809265577483,
 2022: 8.255228706723798,
 2023: 9.533078486705547,
 2024: 9.525957321098034}

# MODIS

In [13]:
def M11_scale_to_celsius(modis_image: ee.image):
   scaled_lsts = modis_image.select(['LST_Day_1km', 'LST_Night_1km']).multiply(0.02).subtract(273.15).rename(['LST_Day', 'LST_Night'])
   return modis_image.addBands(srcImg=scaled_lsts)

def M21_scale_to_celsius(modis_image: ee.Image):
   scaled_lsts = modis_image.select(['LST_Day', 'LST_Night']).subtract(273.15)
   return modis_image.addBands(srcImg=scaled_lsts, overwrite=True)

m11_terra = ee.ImageCollection('MODIS/061/MOD11A2').map(lambda img: M11_scale_to_celsius(img))
m21_terra = ee.ImageCollection('MODIS/061/MOD21C2').map(lambda img: M21_scale_to_celsius(img))
m11_aqua = ee.ImageCollection('MODIS/061/MYD11A2').map(lambda img: M11_scale_to_celsius(img))
m21_aqua = ee.ImageCollection('MODIS/061/MYD21C2').map(lambda img: M21_scale_to_celsius(img))

m11_coll = m11_terra.merge(m11_aqua)
m21_coll = m21_terra.merge(m21_aqua)

In [14]:
def get_average_lst_modis(modis_coll: ee.ImageCollection, start_time, end_time):
   if(modis_coll.size() == 0):
      return None
   mean_temp = modis_coll.filterDate(start_time, end_time).mean()
   day_night_temp = mean_temp.expression("(b('LST_Day') + b('LST_Night')) / 2").rename('LST')
   temp = day_night_temp.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return temp.getInfo()['LST']

In [15]:
lst_m21 = {}
lst_m11 = {}

for year in range(modis_start_year, end_year):
   start_date = ee.Date(f"{year}-{openwater_month_start}-1")
   end_date = ee.Date(f"{year}-{openwater_month_end}-1")
   
   result_m11 = get_average_lst_modis(m11_coll, start_date, end_date)
   # print(f"Complete M11 in {start_date} -- {end_date}")
   result_m21 = get_average_lst_modis(m21_coll, start_date, end_date)
   # print(f"Complete M21 in {start_date} -- {end_date}")
   if(result_m11 is not None):
      lst_m11[year] = result_m11
   if(result_m21 is not None):
      lst_m21[year] = result_m21

print(lst_m11, lst_m21)

{2001: 8.518391334434849, 2002: 10.562313431334651, 2003: 7.857616372707389, 2004: 7.395629688897222, 2005: 9.141430659266959, 2006: 7.007350175325123, 2007: 9.944513991132641, 2008: 8.258408651177758, 2009: 8.258186946255549, 2010: 7.098301657039378, 2011: 8.083154459303564, 2012: 9.068294740221727, 2013: 8.22769966377824, 2014: 10.313111054749465, 2015: 10.214666106018871, 2016: 10.278019374073173, 2017: 9.11163196791068, 2018: 9.083092106624829, 2019: 9.750912104506595, 2020: 10.67362939645613, 2021: 8.178589646995643, 2022: 8.422860681650375, 2023: 10.092309299156064, 2024: 9.718802938390988} {2001: 6.523447096502005, 2002: 11.087026927344535, 2003: 8.22719098563641, 2004: 7.918499190633716, 2005: 9.83460719055784, 2006: 7.375761008911376, 2007: 10.603713281690435, 2008: 8.73401405170392, 2009: 8.932528225671891, 2010: 7.714528772534412, 2011: 8.775993993759588, 2012: 9.7767092369562, 2013: 8.832953221423258, 2014: 10.90875064506874, 2015: 11.124346848601935, 2016: 11.0724993132545

In [10]:
lst_m11

{2001: 8.518391334434845,
 2002: 10.562313431334651,
 2003: 7.857616372707389,
 2004: 7.395629688897221,
 2005: 9.14143065926696,
 2006: 7.007350175325124,
 2007: 9.94451399113264,
 2008: 8.258408651177756,
 2009: 8.258186946255545,
 2010: 7.098301657039377,
 2011: 8.083154459303568,
 2012: 9.068294740221727,
 2013: 8.22769966377824,
 2014: 10.313111054749466,
 2015: 10.214666106018871,
 2016: 10.278019374073171,
 2017: 9.111631967910679,
 2018: 9.083092106624825,
 2019: 9.750912104506595,
 2020: 10.673629396456128,
 2021: 8.178589646995642,
 2022: 8.422860681650373,
 2023: 10.092309299156067,
 2024: 9.718802938390988}

In [11]:
lst_m21

{2001: 6.523447096502005,
 2002: 11.087026927344537,
 2003: 8.227190985636412,
 2004: 7.918499190633716,
 2005: 9.834607190557838,
 2006: 7.375761008911375,
 2007: 10.603713281690437,
 2008: 8.734014051703918,
 2009: 8.932528225671893,
 2010: 7.71452877253441,
 2011: 8.775993993759588,
 2012: 9.776709236956195,
 2013: 8.83295322142326,
 2014: 10.90875064506874,
 2015: 11.124346848601931,
 2016: 11.072499313254557,
 2017: 9.697003172751204,
 2018: 9.721762029880114,
 2019: 10.577461115660952,
 2020: 11.42118506104802,
 2021: 8.943680470222272,
 2022: 9.072915370205486,
 2023: 11.284890175973027,
 2024: 10.757315581774394}

In [ ]:
lst_m11_s = {2001: 8.518391334434845,
 2002: 10.562313431334651,
 2003: 7.857616372707389,
 2004: 7.395629688897221,
 2005: 9.14143065926696,
 2006: 7.007350175325124,
 2007: 9.94451399113264,
 2008: 8.258408651177756,
 2009: 8.258186946255545,
 2010: 7.098301657039377,
 2011: 8.083154459303568,
 2012: 9.068294740221727,
 2013: 8.22769966377824,
 2014: 10.313111054749466,
 2015: 10.214666106018871,
 2016: 10.278019374073171,
 2017: 9.111631967910679,
 2018: 9.083092106624825,
 2019: 9.750912104506595,
 2020: 10.673629396456128,
 2021: 8.178589646995642,
 2022: 8.422860681650373,
 2023: 10.092309299156067,
 2024: 9.718802938390988}

lst_m21_s = {2001: 6.523447096502005,
 2002: 11.087026927344537,
 2003: 8.227190985636412,
 2004: 7.918499190633716,
 2005: 9.834607190557838,
 2006: 7.375761008911375,
 2007: 10.603713281690437,
 2008: 8.734014051703918,
 2009: 8.932528225671893,
 2010: 7.71452877253441,
 2011: 8.775993993759588,
 2012: 9.776709236956195,
 2013: 8.83295322142326,
 2014: 10.90875064506874,
 2015: 11.124346848601931,
 2016: 11.072499313254557,
 2017: 9.697003172751204,
 2018: 9.721762029880114,
 2019: 10.577461115660952,
 2020: 11.42118506104802,
 2021: 8.943680470222272,
 2022: 9.072915370205486,
 2023: 11.284890175973027,
 2024: 10.757315581774394}

# ERA5

In [15]:
def ERA5_scale_to_celsius(era5_image: ee.Image):
   scaled_lsts = era5_image.select(["mean_2m_air_temperature"]).subtract(273.15).rename(["LST"])
   return era5_image.addBands(srcImg=scaled_lsts, overwrite=True)

era5_coll = ee.ImageCollection("ECMWF/ERA5/MONTHLY").map(lambda img: ERA5_scale_to_celsius(img))

In [16]:
def get_average_lst_era5(era5_coll: ee.ImageCollection, start_time, end_time):
   if(era5_coll.size() == 0):
      return None
   mean_temp = era5_coll.filterDate(start_time, end_time).mean()
   temp = mean_temp.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return temp.getInfo()['LST']

In [17]:
lst_era5 = {}

for year in range(landsat_start_year, end_year):
   start_date = ee.Date(f"{year}-{openwater_month_start}-1")
   end_date = ee.Date(f"{year}-{openwater_month_end}-1")
   
   result = get_average_lst_era5(era5_coll, start_date, end_date)
   if(result is not None):
      lst_era5[year] = result
      
lst_era5

{1996: 9.450155880882175,
 1997: 10.306804804710536,
 1998: 10.124025263851733,
 1999: 9.953970489104826,
 2000: 10.897080923980255,
 2001: 11.237577393579986,
 2002: 11.02499017361202,
 2003: 10.489261758126426,
 2004: 10.945637833437745,
 2005: 11.244417451457139,
 2006: 10.083577737021876,
 2007: 11.765040295289413,
 2008: 11.061690752218151,
 2009: 10.35989784117505,
 2010: 10.440376515999297,
 2011: 11.403798160090886,
 2012: 10.77953644155891,
 2013: 9.971784275350128,
 2014: 11.108825121667898,
 2015: 12.168882374570854,
 2016: 11.078814723490853,
 2017: 10.931184669283283,
 2018: 11.564613195835445,
 2019: 11.313782876770295,
 2020: 10.744542470002987}

In [ ]:
lst_era5_s = { 1996: 9.450155880882175,
 1997: 10.306804804710536,
 1998: 10.124025263851733,
 1999: 9.953970489104826,
 2000: 10.897080923980255,
 2001: 11.237577393579986,
 2002: 11.02499017361202,
 2003: 10.489261758126426,
 2004: 10.945637833437745,
 2005: 11.244417451457139,
 2006: 10.083577737021876,
 2007: 11.765040295289413,
 2008: 11.061690752218151,
 2009: 10.35989784117505,
 2010: 10.440376515999297,
 2011: 11.403798160090886,
 2012: 10.77953644155891,
 2013: 9.971784275350128,
 2014: 11.108825121667898,
 2015: 12.168882374570854,
 2016: 11.078814723490853,
 2017: 10.931184669283283,
 2018: 11.564613195835445,
 2019: 11.313782876770295,
 2020: 10.744542470002987 }